# TP 2: Bayesian Inversion for the Aquifer-River Exchange


In this lab, we will perform an inversion of the heat transfer model in a porous medium under transient conditions based on a reference simulation sampled at 4 points. The heat transfer occurs in a portion of a one-dimensional hyporheic zone of 5 meters in height, represented by 250 cells. The difference in head between the top and bottom of the column is set to ±5 cm.

The inversion will focus on the three uncertain parameters of this model, namely:

- Intrinsic permeability ($ k $)
- Thermal conductivity of the solid ( $\lambda_s $)
- Porosity ( $\omega$)
- Heat capacity ( $C$)

We will assume (prior distribution) that these variables are independent and follow the respective distributions:

- $ -\log_{10} k \sim \mathcal{U}(10, 15)$
- $ \lambda_s \sim \mathcal{U}(1, 5) $
- $ \omega \sim \mathcal{U}(0.01, 0.25) $ Pour les simulations on fixera $\omega = 0.05$ 
- $ C \sim \mathcal{U}(1500, 3500) $

The measurement points are located along the column at positions 10 cm, 20 cm, 30 cm, and 40 cm.




In [ ]:
import sys
import os

# Ajoutez le chemin au sys.path
sys.path.append('/home/nradic/Documents/cours_hydro/st4/ginette_code/application/2024_TD_ENS/src_python')



# Listez les fichiers dans le répertoire pour vérifier que Init_folders.py est présent
print("\nFichiers dans le répertoire src_python :")
print(os.listdir('/home/nradic/Documents/cours_hydro/st4/ginette_code/application/2024_TD_ENS/src_python/'))

# Importez les modules
import numpy as np
from pathlib import Path
import pandas as pd
from scipy import interpolate
import matplotlib.pyplot as plt
import matplotlib as mpl
from IPython.display import display
import subprocess
import glob
import itertools
import seaborn as sns
import array

from Init_folders import prepare_ginette_directories, compile_ginette
import Direct_model
from Read_obs import process_obs_data
import Plot
import importlib
importlib.reload(Direct_model)
importlib.reload(Plot)
from Direct_model import setup_ginette, generate_zone_parameters, initial_conditions, boundary_conditions, run_direct_model, run_direct_model_silence, remove_first_two_days
from Plot import plot_obs, plot_obs_zoom, plot_compare_temperatures_obs_sim, plot_temperatures_profiles, plot_water_profiles_interpol,plot_temperature_profiles_interpol,plot_fluxes_timeseries,plot_domain,plot_temperatures_sim,plot_heat_flux_profiles_interpolated
prepare_ginette_directories('/home/nradic/Documents/cours_hydro/st4/ginette_code/application/2024_TD_ENS/GINETTE_SENSI/')

## 1. Field Data

Téléchargement des données observées

In [ ]:

temp_observee = pd.read_csv("../../../../field_data/temp_air_aquifere.dat") 
hydraulic_head_observe = pd.read_csv('../../../../field_data/hydraulic_head.dat')

d_observee = pd.read_csv('')

In [ ]:
temp_observee,hydraulic_head_observe

Représentation des données

In [ ]:
#Représentation Hydraulic head
plt.figure(figsize=(15,5))
plt.plot(hydraulic_head_observe['time']/86400, hydraulic_head_observe['h_top'], label='h_top')
plt.plot(hydraulic_head_observe['time']/86400, hydraulic_head_observe['h_bottom'], label='h_bottom')
plt.xlabel('days')
plt.ylabel('hydraulic head [m]')
plt.legend()
plt.show()

In [ ]:
#Représentation du scénario de température
plt.figure(figsize=(8, 5))
plt.plot(temp_observee['time']/86400, temp_observee['T_top'], label='Top Temp', color='b', lw=2)
plt.plot(temp_observee['time']/86400, temp_observee['T_bottom'], label='Bottom Temp', color='r', ls='--', lw=2)

plt.xlabel('Days', fontsize=12)
plt.ylabel('Temperature (°C)', fontsize=12)
plt.title('Boundary Temperatures', fontsize=14)
plt.legend(fontsize=10)
plt.grid(True)

plt.show()

In [ ]:
#Représentation des données de terrain
plot_temperatures_sim(d_observee, fontsize=15)

## 2. Premier pas dans la résolution du problème inverse

### 2.1 Physical constants

The physical constants used by `Ginette` are $\lambda$ in W/m&deg;C, and $C_s$ in $m^2$ $s^{-2}$ $C^{-1}$, as defined above.

### Intrinsic Permeability [m²]

$$
k = \frac{K \cdot \mu}{\rho \cdot g}
$$

Where:
- $ k $ = intrinsic permeability [m²]
- $ K $ = hydraulic conductivity [m/s]

#### Constant Parameters

- $ \mu $ = viscosity [Pa·s]
- $ \rho $ = density [kg/m³]
- $ g $ = gravitational acceleration [m/s²], approximately \( 9.81 \)

#### Calculation of Intrinsic Permeability
$$
k = 10^{\text{REF}_k}
$$

### Calculation of Thermal Capacity in Ginette Using the Following Relation:

$$
c_{pm} = c_w \cdot \rho_w \cdot n \cdot sat + c_s \cdot r \cdot (1 - \omega) + c_a \cdot \rho_a \cdot n \cdot (1 - sat)
$$

Where:

- $c_s$ is the specific heat capacity of the solid, with the value $\text{val}_c$ in m²/s²/°C. It is recommended to keep this value constant 1000 (kg/m³) because it is not possible to calibrate both parameters, $\rho$ and $c$, simultaneously.
- $J$ is in units of [kg m² s⁻²]
- $c_w = 4185 \, \text{m}^2/\text{s}^2/°C$
- $\rho_w$ represents density in kg/m³.
- $\rho_s$ is the density of solid grains, $[\text{kg/m}^3]$.



### 2.2 Comparaison avec un couple de données

Reprise du code de problème direct

In [ ]:
# Maillage de la modélisation
z_top = 0 # meters
z_bottom = -5 # meters
az=abs(z_top-z_bottom)
n_depths = 250


#discretisation : size cell in meter
dz=az/n_depths
print('dz=',dz)
dz_obs=0.1

# Paramètre de modélisation pour Ginette
# 10 minute interval
dt = 600 # seconds
# 20 days
nb_day = 20# days
state=1
date_simul_bg=pd.to_datetime("2022/04/21 14:00:00")

z_obs=setup_ginette(dt, state, nb_day, z_top, z_bottom, az, dz, date_simul_bg,dz_obs)

# Définition des paramètre du modèle que nous allons simuler
# On considère le sol comme homogène
nb_zone=1
alt_thk=0


#### constant parameters
REF_n = 0.05 # \omega
mu = 1e-6  # Dynamic viscosity in Pa.s 
rho = 1000  # Density in kg/m^3 
g = 9.81  # Acceleration due to gravity in m/s^2
REF_r = 3500


#### unknown parameter

# k=10**REF_k
# REF_k = ???? # m2
# REF_l = ???? # W/m deg C


In [ ]:
# Les conditions aux limites du modèle
obs_temp = 

z_obs=[-5]
initial_conditions(obs_temp, z_top, z_bottom, dz, z_obs)
boundary_conditions(obs_temp,dt)

In [ ]:
# Encadrement des paramètre à determiner


# Choix des valeurs

REF_k = #A completer
REF_l = 

In [ ]:
d_simule = run_direct_model(date_simul_bg,z_bottom, dz, nb_zone, alt_thk, REF_k, REF_n, REF_l, REF_r, REF_k2=None, REF_n2=None, REF_l2=None, REF_r2=None) 

Comparaison avec données de terrain

In [ ]:
plot_compare_temperatures_obs_sim(d_simule,d_observee)

In [ ]:
# Fonction coût


## 3. Automatisation du problème

Etape hors de la boucle

Etape à automatiser

## 4. Représentation des résultats

In [ ]:
#Représentation 2D

In [ ]:
#Histogramme